# 0. Introduction

The aim of this lab if to familiarize students with Dimensionality Reduction and Principle Componenets Analysis (PCA).

# 1. PCA
For this lab we will use the MNIST dataset.

In [ ]:
import torch
from torch.utils import data
import torchvision
from torchvision import datasets

import numpy as np

from sklearn import decomposition
from sklearn import datasets as sk_datasets
import matplotlib.pyplot as plt

from IPython import display

import typing
%matplotlib inline

The MNIST dataset contains 70,000 images of handwritten digits. The images are grayscale, 28x28 pixels and centered.

In [ ]:
mnist_trainset = datasets.MNIST(
    root='./data', 
    train=True, 
    download=True, 
    transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])
    )
mnist_testset = datasets.MNIST(
    root='./data', 
    train=False, 
    download=True, 
    transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),
                               torchvision.transforms.Normalize(
                                 (0.1307,), (0.3081,))
                             ])
    )

In [ ]:
train_loader = data.DataLoader(mnist_trainset, batch_size=512, shuffle=True, drop_last=True)
test_loader = data.DataLoader(mnist_testset, batch_size=512, drop_last=True)

#visualize 10 samples
images, _ = train_loader.__iter__().__next__()
plt.figure()
f, ax = plt.subplots(1,10)
for i in range(10):
    ax[i].imshow(images[i].reshape(28,28), cmap="gray")

As each image is has dimensions 28x28, this results in a 784 vector. However not all pixels are equal. We want to reduce this vector to smaller subset, with the minimum information loss.

We will use sklearn [IncrementalPCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.IncrementalPCA.html) method for this.

Incremental PCA implements the usual Linear PCA analysis, but allows for batch processing.

We will reduce the number of components to 3.

In [ ]:
pca_model = decomposition.IncrementalPCA(n_components=3)

for images, _ in train_loader:
    pca_model.partial_fit(images.reshape(512, -1))

This now allows us to plot the images in a 3D space.

In [ ]:
test_3d = list()
labels_3d = list()
for images, labels in test_loader:
    test_3d.append(
        pca_model.transform(images.reshape(512, -1))
    )
    labels_3d.append(labels)
test_3d, labels_3d = np.stack(test_3d), np.stack(labels_3d)

def get_cmap(n, name='hsv'):
    return plt.cm.get_cmap(name, n)

cmap = get_cmap(10)
fig = plt.figure(figsize=(12, 12))
ax = fig.add_subplot(projection='3d')
for i in range(10):
    idx = labels_3d == i
    ax.scatter(
        test_3d[idx, 0], test_3d[idx, 1], test_3d[idx, 2], # x,y,z
        c=cmap(i),
        label=i
    )
plt.legend()
plt.show()

By plotting the test set in a 3D scatter plot, we can see that images of the same digit are roughly clustered together. This is an indication that some information on the digits is not lost, however we need to better understand the fit we should examine the explained variance.

Read the documentation of IncrementalPCA and print the explained variance and explained variance ratio. Do you think that 3 components are sufficient to represent each image? Would you say that this is a good representation of the original 784 element vector?

Experiment with different number of components to explain 80% of the variance.

In [ ]:
### your code here


Recall from the lectures, to solve PCA we need the eigenvectors of
the covariance matrix so that: $[U,V]=eig(X,X.T)$. However, faster convergence is achieved without using explicit covariance so:  $[U,S,V]=svd(X)$

where:
* Rows of $U$ are the basis
* Diagonal of $S$ are the eigenvalues
* Pick the first $k$ rows

Pytorch provides a lower level method that allows us to explore $[U, S, V]$ through [pca_lowrank](https://pytorch.org/docs/stable/generated/torch.pca_lowrank.html)


In [ ]:
for images, _ in train_loader:
    (U, S, V) = torch.pca_lowrank(images.reshape(512, -1), q=100, center=True, niter=10)
U.shape, S.shape, V.shape

$A$ is a data matrix with m samples and n features

$V$ columns represent the principal directions

$S^2 / (m - 1)$ contains the eigenvalues of $A.T * A / (m - 1)$

`matmul(A, V[:, :k])` projects data to the first k principal components.

In [ ]:
images, _ = test_loader.__iter__().__next__()
images = images.reshape(512, -1)
n_components = 3 # experiment with different number of components
pca_matrix = torch.matmul(images, V[:, :n_components])
pca_matrix.shape

we can also reconstruct the images by reversing the operation

In [ ]:
reconstructed_images =  torch.matmul(V[:, :n_components], pca_matrix.T).T
reconstructed_images.shape

Now visualising examples from the two sets we can also visually inspect the quality of the reconstruction. Does the reconstruction match your expectations?

In [ ]:
f, ax = plt.subplots(2,10)
for j in range(2):
    for i in range(10):
        if j == 0:
            ax[j][i].imshow(images[i].reshape(28,28), cmap="gray") # top row shows raw images
        else:
            ax[j][i].imshow(reconstructed_images[i].reshape(28,28), cmap="gray") # bottom row shows reconstructed

## 1.1 PCA in practice

Now that we have seen how PCA works and have assessed it quantitatively and qualititively, we will train a classifier on the MNIST dataset.

First, using pytorch built in methods train a classification MLP model on the raw MNIST image vectors, with number of inputs equal to the 784 (number of pixels), 512 nodes in the hidden layer and 10 nodes in the output layer.

In [ ]:
### your code here

Estimate the accuracy of the classifier on the test set.

In [ ]:
### your code here

Now train another classifier on the images reduced using PCA and calculate the classifiers accuracy. How do the two classifiers compare?

In [ ]:
### your code here

# 2. Feature Selection

For this part of the lab we will use the [diabetes](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html) dataset.

In [ ]:
diabetes_db = sk_datasets.load_diabetes()
diabetes_db.data.shape

In week 3, we used all features from this dataset to estimate diabetes however in this case we will use the feature selection strategies from the lecture to filter the attributes.

First calculate the correlation coefficient of each attribute against Y. Select the attributes with the 10% highest **absolute** coeficient and train a Linear Regression model on them, using pytorch built in methods. 

In [ ]:
### your code here

Repeat the feature selection method for  $χ^2$ statistical independence and repeat the training process.

Hint: check scipy package

In [ ]:
### your code here